##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Grundlegende Klassifizierung: Bilder von Kleidung bestimmen

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/classification"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/classification.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In diesem Guide wird ein neuronales Netzwerkmodell trainiert, um Bilder von Kleidung, wie Turnschuhe und Hemden, zu klassifizieren. Wenn du nich alle Details verstehst ist das nicht schlimm. Dies ist ein kurzer Überblick über ein komplettes TensorFlow-Programm, bei dem die Details nach und nach erklärt werden.

Es wird [tf.keras](https://www.tensorflow.org/guide/keras) als API verwendet.

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

## Daten importieren (Fashion MNIST)

Wir verwenden den [Fashion MNIST](https://github.com/zalandoresearch/fashion-mnist) Datensatz in welchem 70,000 schwarz-weiß Bilder in 10 Kategorien eingeteilt sind. die Bilder zeigen einzelne Kleidungsstücke in niedriger Auflösung (28 x 28 Pixel):

<table>
  <tr><td>
    <img src="https://tensorflow.org/images/fashion-mnist-sprite.png"
         alt="Fashion MNIST sprite"  width="600">
  </td></tr>
  <tr><td align="center">
    <b>Bild 1.</b> <a href="https://github.com/zalandoresearch/fashion-mnist">Fashion-MNIST Beispiel</a> (by Zalando, MIT License).<br/>&nbsp;
  </td></tr>
</table>

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

Die Bilder sind eingeteilt in *Labels*. Sie entsprechen der *Klasse* der Kleidung, welches das Bild darstellt:

<table>
  <tr>
    <th>Label</th>
    <th>Class</th>
  </tr>
  <tr>
    <td>0</td>
    <td>T-shirt/top</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Trouser</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Pullover</td>
  </tr>
    <tr>
    <td>3</td>
    <td>Dress</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Coat</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Sandal</td>
  </tr>
    <tr>
    <td>6</td>
    <td>Shirt</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Sneaker</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Bag</td>
  </tr>
    <tr>
    <td>9</td>
    <td>Ankle boot</td>
  </tr>
</table>

Jedes Bild ist einer Kategorie(Label) zugeordnet.

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## Daten erkunden

Schauen wir uns das Format des Datensatzes vor dem Training des Modells an. Die folgende Abbildung zeigt, dass der Trainingsdatensatz 60.000 Bilder enthält, wobei jedes Bild mit 28 x 28 Pixeln dargestellt wird:

In [ ]:
train_images.shape

Ebenso gibt es 60.000 Labels in den Trainingsdaten:

In [ ]:
len(train_labels)

Jedes Label ist einer Zahl von 0 bis 9 zugeordnet:

In [ ]:
train_labels

Der Testsatz besteht aus 10.000 Bildern. Auch hier wird jedes Bild als 28 x 28 Pixel dargestellt:

In [ ]:
test_images.shape

Und der Testsatz enthält 10.000 Bildbeschriftungen:

In [ ]:
len(test_labels)

## Daten vorbereiten

Die Daten müssen vor dem Training des Netzes aufbereitet werden. Wenn du das erste Bild im Trainingssatz untersuchst, wirst du feststellen, dass die Pixelwerte in den Bereich von 0 bis 255 fallen:

In [ ]:
plt.figure()
plt.imshow(train_images[0])
plt.colorbar()
plt.grid(False)
plt.show()

Skaliere diese Werte auf einen Bereich von 0 bis 1, bevor du sie in das neuronale Netz eingibst. Teile dazu die Werte durch 255. Es ist wichtig, dass der Trainingssatz und der Testsatz auf dieselbe Weise vorverarbeitet werden:

In [ ]:
train_images = train_images / 255.0

test_images = test_images / 255.0

Um zu überprüfen, ob die Daten im richtigen Format vorliegen, zeigen wir die ersten 25 Bilder aus dem Trainingssatz an und geben den Klassennamen unter jedem Bild an.

In [ ]:
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Model erstellen

Damit wir das neuronale Netz erstellen können, müssen wir zuerst die einzelnen Schichten definieren.

### Ebenen konfigurieren

Der Basis eines neuronalen Netzes ist ein [*Layer*] (https://www.tensorflow.org/api_docs/python/tf/keras/layers). Diese Schichten verarbeiten verschiedene Informationen der Daten. 

Der Großteil des Deep Learning besteht aus der Verkettung einfacher Schichten. Die meisten Schichten, wie z. B. `tf.keras.layers.Dense`, ermöglichen das Lernen während des Trainings.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10)
])

Die erste Schicht in diesem Netzwerk, tf.keras.layers.Flatten, wandelt das Format der Bilder von einem zweidimensionalen Array (mit 28 mal 28 Pixeln) in ein eindimensionales Array (mit 28 * 28 = 784 Pixeln) um. Stelle dir diese Ebene so vor, dass die Pixelreihen im Bild auseinander genommen und aneinandergereiht werden. Diese Ebene formatiert die Daten neu.

Nachdem die Pixel "geglättet" wurden, besteht das Netz aus einer Folge von zwei "tf.keras.layers.Dense"-Schichten. Dies sind verbundene neuronale Schichten. Die erste "Dense"-Schicht hat 128 Knoten (oder Neuronen). Die zweite (und letzte) Schicht gibt ein logits-Array mit einer Länge von 10 zurück. Jeder Knoten enthält eine Punktzahl, die angibt, dass das aktuelle Bild zu einer der 10 Klassen gehört.

### Erstelle das Modell

Bevor das Modell für das Training bereit ist, benötigt es noch ein paar Einstellungen. Diese werden bei der Erstellung des Modells hinzugefügt  [*compile*](https://www.tensorflow.org/api_docs/python/tf/keras/Model#compile) step:

* [*Loss function*](https://www.tensorflow.org/api_docs/python/tf/keras/losses) —Damit wird gemessen, wie genau das Modell während des Trainings ist. Wir möchten diese Funktion minimieren, um das Modell in die richtige Richtung zu "lenken"..
* [*Optimizer*](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) —Dient der Verbesserung des Modells.
* [*Metrics*](https://www.tensorflow.org/api_docs/python/tf/keras/metrics) —Dient zur Überwachung der Trainings- und Testschritte. Im folgenden Beispiel wird die *Genauigkeit* verwendet, d. h. der Anteil der Bilder, die korrekt klassifiziert wurden.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Trainiere das Modell

Das Training des neuronalen Netzmodells erfordert folgende Schritte:

1. Gebe die Trainingsdaten an das Modell weiter. In diesem Beispiel befinden sich die Trainingsdaten in den Arrays `train_images` und `train_labels`.
2. Das Modell lernt, Bilder und Bezeichnungen zuzuordnen.
3. Das Model beginnt Vorhersagen über einen Testsatz zu machen - in diesem Beispiel das Array "test_images".
4. Überprüfe, ob die Vorhersagen mit den Bezeichnungen aus dem Array "test_labels" übereinstimmen.


### Fütter das Modell mit Daten

Rufe [`model.fit`](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) auf, damit das Training gestartet wird.

In [ ]:
model.fit(train_images, train_labels, epochs=10)

Während das Modell trainiert, werden die Verlust- und Genauigkeitsmetriken angezeigt. Dieses Modell erreicht eine Genauigkeit von etwa 0,91 (oder 91 %) bei den Trainingsdaten.

### Bewerte die Genauigkeit

Vergleiche anschließend, wie das Modell im Testdatensatz abschneidet:

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

Es stellt sich heraus, dass die Genauigkeit im Testdatensatz etwas geringer ist als die Genauigkeit im Trainingsdatensatz. Diese Diskrepanz zwischen Trainingsgenauigkeit und Testgenauigkeit stellt eine Überanpassung dar. Eine Überanpassung liegt vor, wenn ein Modell für maschinelles Lernen bei neuen, zuvor unbekannten Eingaben schlechter abschneidet als bei den Trainingsdaten. Ein überangepasstes Modell "merkt" sich das Rauschen und die Details im Trainingsdatensatz bis zu einem Punkt, an dem es sich negativ auf die Leistung des Modells bei den neuen Daten auswirkt. Weitere Informationen finden Sie im Folgenden:

*   [Überanpassung/Overfitting](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit#demonstrate_overfitting)
*   [Vermeidung von Überanpassung](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit#strategies_to_prevent_overfitting)

### Treffe Vorhersagen

Nachdem das Modell trainiert wurde, können wir damit Vorhersagen über einige Bilder treffen.
Füge eine Softmax-Schicht hinzu, um die linearen Ausgaben des Modells - [logits](https://developers.google.com/machine-learning/glossary#logits) - in Wahrscheinlichkeiten umzuwandeln, die leichter zu interpretieren sein sollten.

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(test_images)

Hier hat das Modell die Bezeichnung für jedes Bild in der Testgruppe vorhergesagt. Werfen wir einen Blick auf die erste Vorhersage:

In [ ]:
predictions[0]

Eine Vorhersage ist eine Reihe von 10 Zahlen. Sie stellen die "Zuversicht" des Modells dar, dass das Bild mit jedem der 10 verschiedenen Kleidungsstücke übereinstimmt. Wir können sehen, welches Etikett den höchsten Vertrauenswert hat:

In [ ]:
np.argmax(predictions[0])

Das Modell ist also sehr zuversichtlich, dass es sich bei diesem Bild um eine Stiefelette handelt, oder `class_names[9]`. Die Prüfung des Testetiketts zeigt, dass diese Klassifizierung richtig ist:

In [ ]:
test_labels[0]

Stelle dies grafisch dar, um den vollständigen Satz von 10 Klassenvorhersagen zu betrachten.

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

### Vorhersagen überprüfen

With the model trained, you can use it to make predictions about some images.
Nun können wir das Modell Vorhersagen treffen lassen. Wir prüfen ob unser Model anhand eines neuen Bildes sagen kann, was für ein Kleigundsstück es ist.

Schauen wir uns das erste Bild an (Nummer 0). Richtige Vorhersagen sind blau und falsche Vorhersagen sind rot. Die Zahl gibt den Prozentsatz (von 100) für die Vorhersage an.

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images)
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

Stellen wir mehrere Bilder mit ihren Vorhersagen dar. Beachte, dass das Modell auch dann falsch sein kann, wenn es sehr zuverlässig ist.

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

## Verwende das trainierte Modell

Schließlich verwenden wir das trainierte Modell, um eine Vorhersage für ein einzelnes Bild zu treffen.

In [ ]:
# Grab an image from the test dataset.
img = test_images[1]

print(img.shape)

Mit `tf.keras` können wir auch viele Bilder auf einmal verarbeiten.

In [ ]:
# Add the image to a batch where it's the only member.
img = (np.expand_dims(img,0))

print(img.shape)

Jetzt bestimmen wir die Kategorie

In [ ]:
predictions_single = probability_model.predict(img)

print(predictions_single)

In [ ]:
plot_value_array(1, predictions_single[0], test_labels)
_ = plt.xticks(range(10), class_names, rotation=45)
plt.show()

`tf.keras.Model.predict` gibt eine Liste von Listen zurück - eine Liste für jedes Bild im Datenstapel. Erfasse die Vorhersagen für unser (einziges) Bild im Stapel:

In [ ]:
np.argmax(predictions_single[0])